## 2D attempt at solving the bidomain equations:

 mpirun -n 2 python3 2D_bidomain.py
- naredba za pokretanje programa na 2 procesora (prebaciti u .py)

In [1]:
from dolfinx.mesh import locate_entities, create_submesh
from dolfinx.fem import (
    Constant,
    Function,
    FunctionSpace,
    VectorFunctionSpace,
    Expression
)
from ufl import (
    TestFunctions,
    dx,
    grad,
    inner,
    dot,
    Identity,
    outer,
    Measure,
    SpatialCoordinate,
    lt,
    conditional,
    
)

from configs import *
from utils import *
import numpy as np
from tqdm import tqdm
from mpi4py import MPI

### Generating a mesh:

In [2]:
# Create a mesh
mh = MeshHandler()
Omega = mh.create(N=N)

x = SpatialCoordinate
cells_H = locate_entities(Omega, Omega.topology.dim, Omega_H)
cells_T = locate_entities(Omega, Omega.topology.dim, Omega_T)

Omega_Heart, _, vertices_H, geom_H = create_submesh(Omega, Omega.topology.dim, cells_H)
Omega_Torso, _, vertices_T, geom_T = create_submesh(Omega, Omega.topology.dim, cells_T)

# mh.plot_subdomains(Omega, cells_H, cells_T)
# mh.plot_mesh(Omega)

### Bidomain model:

In [3]:
# Define function space
V2 = VectorFunctionSpace(Omega, ("CG", 2), 2)
V1 = FunctionSpace(Omega, ("CG", 2))

# Define test functions
phi, psi = TestFunctions(V2)

# Define functions
v, v_n, v_nn = Function(V2), Function(V2), Function(V2)
w, w_n, w_nn = Function(V1), Function(V1), Function(V1)
I_ion, g, I_app = Function(V1), Function(V1), Function(V1)
fibres = Function(V2)
V_m, u = v.split()
V_m_n, u_n = v_n.split()
V_m_nn, u_nn = v_nn.split()

# V_m, u = v.sub(0), v.sub(1)
# V_m_n, u_n = v_n.sub(0), v_n.sub(1)
# V_m_nn, u_nn = v_nn.sub(0), v_nn.sub(1)

  V2 = VectorFunctionSpace(Omega, ("CG", 2), 2)



In [4]:
# initial conditions
V_m_n.interpolate(lambda x: np.full((x.shape[1],), V_MIN))
w_n.interpolate(lambda x: np.full((x.shape[1],), 1 / (V_MAX - V_MIN) ** 2))
V_m_nn.interpolate(lambda x: np.full((x.shape[1],), V_MIN))
w_nn.interpolate(lambda x: np.full((x.shape[1],), 1 / (V_MAX - V_MIN) ** 2))

# interpolation part
# fibres
fun_fibres = eval_fibres()
fibres.interpolate(fun_fibres)

# ionic current
fun_I_ion = eval_I_ion(v_n, w_n)
I_ion.interpolate(fun_I_ion)

## gating variable
# fun_g = eval_g(v_n, w_n)
# g.interpolate(fun_g)

# applied current
fun_I_app = eval_I_app()
I_app.interpolate(fun_I_app)

In [ ]:
def I():
    return np.exp(
        -(((x[0] - self.Hx) / self.size) ** 2)
        - ((x[1] - self.Hy) / self.size) ** 2(self.t / self.t_act) ** 2
    )

I_app = Expression()

In [5]:
condition = lt(V_m, V_GATE)
true_statement = w / TAU_OPEN - 1 / TAU_OPEN / (V_MAX - V_MIN) ** 2
false_statement = w / TAU_CLOSE
g = conditional(condition, true_statement, false_statement)

In [8]:
# defining UFL language parts
# conductivities
sigma_il = Constant(Omega, SIGMA_IL)
sigma_it = Constant(Omega, SIGMA_IT)
sigma_el = Constant(Omega, SIGMA_EL)
sigma_et = Constant(Omega, SIGMA_ET)
sigma_tlt = Constant(Omega, SIGMA_TLT)

d = Omega.topology.dim

sigma_i = sigma_it * Identity(d) + (sigma_il - sigma_it) * outer(fibres, fibres)
sigma_e = sigma_et * Identity(d) + (sigma_el - sigma_et) * outer(fibres, fibres)
sigma_t = sigma_tlt * Identity(d)

# Constants used in variational forms
dt = Constant(Omega, DT)
A_m = Constant(Omega, A_M)
C_m = Constant(Omega, C_M)

In [10]:
# Define new measures associated with the interior domains and
# exterior boundaries
dx_H = Measure("dx", subdomain_data=Omega_Heart)
dx_T = Measure("dx", subdomain_data=Omega_Torso)

In [12]:
# Define variational problem
F = (
    A_m * (C_m * (V_m - V_m_n) + dt * I_ion) * phi * dx_H
    + dt * inner(dot(sigma_i, grad(V_m + u)), grad(phi)) * dx_H
    - dt * A_m * I_app * phi * dx(1)
)
F += (
    inner(dot(sigma_i + sigma_e, grad(u)), grad(psi)) * dx_H
    + inner(dot(sigma_i, grad(V_m)), grad(psi)) * dx_H
    + inner(dot(sigma_t, grad(u)), grad(psi)) * dx_T
)

In [13]:
print(F)

{ -1 * v_0[0] * f * c_5 * c_6 } * dx(<Mesh #0>[1], {})
  +  { v_0[0] * c_6 * (c_5 * f + c_7 * (f_0 + -1 * f_0)) } * dx(<Mesh #0>[everywhere], {})
  +  { c_5 * (conj(((grad(v_0[0])) : ((({ A | A_{i_8, i_9} = I[i_8, i_9] * c_1 }) + ({ A | A_{i_{10}, i_{11}} = ((f) (X) (f))[i_{10}, i_{11}] * (c_0 + -1 * c_1) })) . (grad(f_0 + f_1)))))) } * dx(<Mesh #0>[everywhere], {})
  +  { conj(((grad(v_0[1])) : ((({ A | A_{i_8, i_9} = I[i_8, i_9] * c_1 }) + ({ A | A_{i_{10}, i_{11}} = ((f) (X) (f))[i_{10}, i_{11}] * (c_0 + -1 * c_1) }) + ({ A | A_{i_{12}, i_{13}} = I[i_{12}, i_{13}] * c_3 }) + ({ A | A_{i_{14}, i_{15}} = ((f) (X) (f))[i_{14}, i_{15}] * (c_2 + -1 * c_3) })) . (grad(f_1))))) } * dx(<Mesh #0>[everywhere], {})
  +  { conj(((grad(v_0[1])) : ((({ A | A_{i_8, i_9} = I[i_8, i_9] * c_1 }) + ({ A | A_{i_{10}, i_{11}} = ((f) (X) (f))[i_{10}, i_{11}] * (c_0 + -1 * c_1) })) . (grad(f_0))))) } * dx(<Mesh #0>[everywhere], {})
  +  { conj(((grad(v_0[1])) : (({ A | A_{i_{16}, i_{17}} = I[i_{16}, i_{17

In [ ]:
from dolfinx.fem.petsc import NonlinearProblem

In [ ]:
# Time-stepping
t = 0
for n in tqdm(range(NUM_STEPS)):
    # Update current time and expressions
    t += DT
    g.v = v
    I_ion.v = v
    I_app.t = t

    # Solve variational problem for time step
    # set_log_active(False)
    problem = NonlinearProblem(F, v)
    u_ = problem.solve()

    # Update previous solution
    v_n.assign(v)